<a href="https://colab.research.google.com/github/JakobSchauser/BachelorProject-IceCube-ML/blob/main/Fourth_Working_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install spektral -q

import numpy as np
import tensorflow as tf
import keras
import pickle

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.metrics import MeanAbsoluteError as MeanAbsoluteError_acc
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader, Dataset
from spektral.datasets import QM9
from spektral.layers import AGNNConv, GlobalSumPool, GCNConv

from tqdm.notebook import tqdm

import wandb
from wandb.keras import WandbCallback

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

PI = np.pi

# Lifehack
true = True
false = False

In [50]:
!pip install wandb -q

import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [51]:
################################################################################
# LOAD DATA
################################################################################

class dat(Dataset):
  def __init__(self,n = 1,**kwargs):
    self.n = n
    super().__init__(**kwargs)
  def read(self):
    # path = "/content/drive/MyDrive/Bachelor Project - IceCube ML/generatedDataAnglesEnergy100000 0.npz" # Uncleaned
    path = "/content/drive/MyDrive/Bachelor Project - IceCube ML/generatedDataAnglesEnergyClean200000events 0.npz" # Cleaned
    # path = "/content/drive/MyDrive/Bachelor Project - IceCube ML/data.dat" # From Severin
    dataset = np.load(path,allow_pickle = True)["arr_0"]
    graphs = []
    for g in dataset:
      graphs.append(g)
    return np.array(dataset)

dataset = dat()

print("Dataset is", dataset, "consisting of", dataset[0])

# Find the reverse transformer
pcl = "/content/drive/MyDrive/Bachelor Project - IceCube ML/transformers.pkl"

with open(pcl,'rb') as tra:
    transformer = pickle.load(tra)

transformer = transformer["truth"]


#### Put on GPU when possible
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print("Running on GPU")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("Running on CPU")

Dataset is dat(n_graphs=200000) consisting of Graph(n_nodes=13, n_node_features=5, n_edge_features=None, n_labels=3)
Running on GPU


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RobustScaler from version 0.19.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [52]:
def custom_loss(target,predictions):
    predictions = tf.cast(predictions,"float64")

    energy    = tf.abs(target[:,0] - predictions[:,0])
    azi, zeni = tf.abs(target[:,1] - predictions[:,1]), tf.abs(target[:,2] - predictions[:,2])


    azi  = tf.minimum(azi, tf.abs(tf.abs(azi )-2))     # 2 should be pi or 2pi
    zeni = tf.minimum(tf.abs(zeni), tf.abs(tf.abs(zeni)-2))  # 2 should be pi or 2pi

    # azi, zeni = azi*10,zeni*10

    #Trying something crazy
    # more = 2
    # menergy, mazi, mzeni = tf.where(energy<tf.abs(predictions[:,0]-mean_energy),energy,energy*more), tf.where(azi<tf.abs(predictions[:,1]-mean_azimuth),azi,azi*more), tf.where(zeni<tf.abs(predictions[:,2]-mean_zenith),zeni,zeni*more)
    # energy,azi,zeni = menergy,mazi,mzeni


    loss = 0
    loss += tf.reduce_mean(tf.square(energy))
    loss += tf.reduce_mean(tf.square(azi))
    loss += tf.reduce_mean(tf.square(zeni))
    loss += 0.1
    return loss


################################################################################
# PARAMETERS
################################################################################

config = {
        "learning_rate": 1e-4,
        "epochs": 50,
        "batch_size": 64*4,
        "loss_function": "custom_loss",
        "architecture": "Broadening CNN",
        "dataset": "MuonGun"
      }

learning_rate = config["learning_rate"] # Learning rate
epochs = config["epochs"]  # Number of training epochs
test_epochs = 2 # Number of testing epochs
batch_size = config["batch_size"] # Batch size


# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split
idxs = np.random.permutation(len(dataset))
split = int(0.9 * len(dataset))
idx_tr, idx_te = np.split(idxs, [split])
dataset_tr, dataset_te = dataset[idx_tr], dataset[idx_te]

train_loader = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
test_loader = DisjointLoader(dataset_te, batch_size=batch_size, epochs=test_epochs, shuffle=True)

In [53]:
network_size = 64 # one-variable network size changer

class model_class(Model):
  def __init__(self):
    super().__init__()
    
    steps = [i**2 for i in range(1,5)]

    # self.inp    = AGNNConv()
    self.inp = GCNConv(network_size)
    self.encode = [GCNConv(network_size*s, activation = "relu") for s in steps]
    self.decode = [Dense(network_size*s, activation = "relu") for s in steps[::-1]]
    self.pool   = GlobalSumPool()
    self.out    = Dense(n_out) 

  def call(self, inputs, training = False):
    x, a, i = inputs
    # e    = self.generate_edge_features(x, a)  NEXT STEP is implementing this
    x = self.inp([x,a])

    for enc in self.encode:
      x = enc([x,a])

    for dec in self.decode:
      x = dec(x)
    
    x = self.pool([x,i])

    x_out = self.out(x)

    return x_out

  def generate_edge_features(x,a):
    send    = a.indices[:, 0]
    receive = a.indices[:, 1]

      # forwards  = tf.gather(x[:, 3], send) <= tf.gather(x[:, 3], receive)

      # send    = tf.cast(send[forwards], tf.int64)
      # receive = tf.cast(receive[forwards], tf.int64)

      # a       = SparseTensor(indices = tf.stack([send, receive], axis = 1), values = tf.ones(tf.shape(send), dtype = tf.float32), dense_shape = tf.cast(tf.shape(a), tf.int64))

    diff_x  = tf.subtract(tf.gather(x, receive), tf.gather(x, send))

    dists   = tf.sqrt(
      tf.reduce_sum(
        tf.square(
          diff_x[:, :3]
        ), axis = 1
      ))

    vects = tf.math.divide_no_nan(diff_x[:, :3], tf.expand_dims(dists, axis = -1))

    e = tf.concat([diff_x[:, 3:], tf.expand_dims(dists, -1), vects], axis = 1)

    return a, e


# Build model
model = model_class()

opt = Adam(lr=learning_rate)
loss_fn = custom_loss
# loss_fn = MeanAbsoluteError()
acc_fn = MeanAbsoluteError()


model.compile()
# model.build()

In [54]:
################################################################################
# DEFINE TF-FUNCTIONS
################################################################################
@tf.function(input_signature=train_loader.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions)
        loss += sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))
    acc = acc_fn(target,predictions)

    return loss, acc

def validate(inputs,target):
    predictions = model(inputs, training=False)
    loss = loss_fn(target, predictions)
    loss += sum(model.losses)
    acc = acc_fn(target,predictions)

    return loss, acc

def scale_inputs(inputs):
    inputs[0][:,:3] = inputs[0][:,:3]/100 # x y z
    inputs[0][:,3] = inputs[0][:,3]/10000 # time
    inputs[0][:,4] = inputs[0][:,4]/1 # charge
    return inputs



loss_fn = custom_loss


def fit(train_loader):
  run = wandb.init(project='IceCube GNN Bachelor',config = config)
  current_batch = 0
  model_loss = []
  model_acc = []
  vali_acc = []
  epoch = 0

  epoch_steps = train_loader.steps_per_epoch

  t = tqdm(total  = train_loader.steps_per_epoch,leave = True)
  t.set_description(f'Currently on epoch {0} of {epochs} ')

  pb = []
  tar = []
  step = 0

  last_vali = 0
  for batch in train_loader:
      step += 1
      inputs, target = batch
      inputs = scale_inputs(inputs)

      loss, acc = train_step(inputs,target)

      model_loss.append(loss)
      model_acc.append(acc)
      current_batch += 1
      t.update(1)
      if current_batch == epoch_steps:
          test_loader = DisjointLoader(dataset_te, batch_size=batch_size, epochs=test_epochs, shuffle=True)
          t.set_description(f'Currently validating results')

          for vali_batch in test_loader:
            vali_inputs, vali_target = vali_batch
            valie_inputs = scale_inputs(vali_inputs)
            vali_loss, va = validate(vali_inputs,vali_target)
            vali_acc.append(vali_loss)
          va = np.mean(vali_acc)
          change = (va-last_vali)/max(0.000001,last_vali)
          last_vali = va
          s = "Train loss: {} - Train accuracy: {} | Validation loss {} - Change {:.3}% | Epoch: {}".format(np.mean(model_loss), np.mean(model_acc), va, change*100, epoch)
          t.write(s)
          if epoch%5 == 0 and epoch != 0:
            print("Two current guesses are\n",np.array(pb[0]),np.array(pb[1]),"for\n",tar[0],tar[1])
          else:
            pb = model(inputs, training=False)
            tar = target


          wandb.log({
                    "train_loss": np.mean(model_loss),
                    "train_accuracy": np.mean(model_acc),
                    "val_loss": va,
                    })




          model_loss = []
          model_acc = []
          vali_acc = []
          current_batch = 0
          step = 0

          t.n = 0
          t.last_print_n = 0
          t.refresh()
          epoch += 1

          t.set_description(f'Currently on epoch {epoch} of {epochs} ')


In [55]:
fit(train_loader)

save_path = "/content/drive/MyDrive/Bachelor Project - IceCube ML/Saved Models"
pth = save_path + "/AnglesEnergy-FourthIteration-50epochs" 
model.save(pth)

train_loss,0.52131
train_accuracy,0.34636
val_loss,0.54125
_runtime,4748
_timestamp,1613491519
_step,55


train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


Train loss: 8229.944621867711 - Train accuracy: 12.855319023132324 | Validation loss 4.419486752006191 - Change 4.42e+08% | Epoch: 0
Train loss: 1866.68025233734 - Train accuracy: 5.464889049530029 | Validation loss 5.483106610544051 - Change 24.1% | Epoch: 1
Train loss: 63.30006269743019 - Train accuracy: 1.5564296245574951 | Validation loss 1.5706824362460736 - Change -71.4% | Epoch: 2
Train loss: 105.14171993249487 - Train accuracy: 1.349956750869751 | Validation loss 5.020337737489885 - Change 2.2e+02% | Epoch: 3
Train loss: 267.6710410789988 - Train accuracy: 2.2290613651275635 | Validation loss 17.43989328549702 - Change 2.47e+02% | Epoch: 4
Train loss: 41.42047787593211 - Train accuracy: 1.2094295024871826 | Validation loss 3.3618958470732037 - Change -80.7% | Epoch: 5
Two current guesses are
 [ 0.9125924  4.896446  -3.6563542] [ 0.20558025  0.33005065 -0.43321952] for
 [ 0.74973726 -0.73528551 -0.28300092] [ 0.01178087 -0.068386   -0.49966352]
Train loss: 19.756627629285642 - T

In [56]:
model.summary()

Model: "model_class_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gcn_conv_25 (GCNConv)        multiple                  320       
_________________________________________________________________
gcn_conv_26 (GCNConv)        multiple                  4096      
_________________________________________________________________
gcn_conv_27 (GCNConv)        multiple                  16384     
_________________________________________________________________
gcn_conv_28 (GCNConv)        multiple                  147456    
_________________________________________________________________
gcn_conv_29 (GCNConv)        multiple                  589824    
_________________________________________________________________
dense_25 (Dense)             multiple                  1049600   
_________________________________________________________________
dense_26 (Dense)             multiple                